# Tracking Batter Eligibility

### MSP 8.28.2017

Goals:
1. Be ready for possible eligibility changes.

In [ ]:
from IPython.display import HTML
HTML('''<script>code_show=true; function code_toggle() { if (code_show){ $('div.input').hide();} else { $('div.input').show(); } code_show = !code_show} $( document ).ready(code_toggle);</script><form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [1]:
# boilerplate imports

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import matplotlib as mpl



In [2]:
# imports for scraping

from bs4 import BeautifulSoup
import requests


# basic birth year reader. tables[1] has good stuff in it!
def get_birth_year(tables):

    birthdate_start = (tables[1].text).find("Birthdate:")

    birthdate_end =  (tables[1].text).find('(',birthdate_start)

    #print (tables[1].text)[birthdate_start:birthdate_end]
    birth_year = (tables[1].text)[birthdate_start:birthdate_end].split('/')[2]
    #print birth_year
    
    return birth_year





#### Compile Hitters

This cell only needs to be run in the event that player ID numbers change, or new players are added.

In [3]:

HDict = {}

f = open('data/batterdict.dat')

for line in f:
    #print line
    try:
        HDict[(line.split('|')[0]).strip()] = line.split('|')[1]
    except:
        pass
    
    
print('Found {} hitters by scraping teams.'.format(len(HDict.keys())))

Found 881 hitters by scraping teams.


### Grab Game Logs

Get the following quantities:
1. Batting Order
2. Position(s) Played

In [58]:
#
# for an individual player
#'

itable = [u'Date', u'Team', u'Opp', u'BO', u'Pos', u'PA', u'H',\
          u'2B', u'3B', u'HR', u'R', u'RBI', u'SB', u'CS',\
          u'BB%', u'K%', u'ISO', u'BABIP', u'AVG', u'OBP', u'SLG', u'wOBA', u'wRC+']
#             0         1        2    3     4      5     6  \
#            7      8   9       10    11      12     13  \
#            14    15    16    17          18       19      20     21      22




plist = ['Joey Votto','Albert Pujols','Mike Trout','Buster Posey','Byron Buxton']

plist = HDict.keys()

#for player in HDict.keys():

def obtain_players(plist):
    
    # set up dictionary
    PVals = {}

    pnum = 0

    for player in plist:
        PVals[player] = {}
        #print player,

        get_url = 'http://www.fangraphs.com/statsd.aspx?playerid='+str(HDict[player].strip())
        
        #print get_url

        r  = requests.get(get_url)

        data = r.text

        soup = BeautifulSoup(data)

        tables = soup.findAll('table')

        birth_year = float(get_birth_year(tables))

        PVals[player]['bo'] = []
        PVals[player]['pos'] = []


        for indx,table in enumerate(tables):

            #print indx,[th.get_text() for th in table.find("tr").find_all("th")]

            if [th.get_text() for th in table.find("tr").find_all("th")] == itable:

                #print get_url
                if (pnum % 50) == 0:
                    print pnum,player

                pnum += 1

                #print [th.get_text() for th in table.find("tr").find_all("th")]

                #print table.find_all("tr", class_="grid_multi")

                for row in table.find_all("tr")[1:]:
                    

                    if (row in table.find_all("tr", class_="grid_postseason")) |\
                        (row in table.find_all("tr", class_="grid_multi")):
                            #print 'Multi and/or Postseason:',sav[0]
                            continue


                    sav = [td.get_text() for td in row.find_all("td")]
                    
                    #print sav
                    
                    #print sav
                
                    
                    if ('Date' not in sav[0]) & ('Total' not in sav[0]) :
                        PVals[player]['bo'].append(float(sav[3]))
                        
                        try:
                            if sav[4][1] == 'F': 
                                PVals[player]['pos'].append('OF')
                        
                            elif sav[4][1] == '-':
                                PVals[player]['pos'].append(sav[4][0:1])
                        
                            else:
                                PVals[player]['pos'].append(sav[4][0:2])
                                
                        except:
                            PVals[player]['pos'].append(sav[4][0:1])

                    #if (np.sum([(x.strip() in sav[1]) for x in projections])==0) & \
                    #(np.sum([(x in sav[1]) for x in minors])==0) & \
                    #(np.sum([(x in sav[0]) for x in agg])==0): 
                    #    try:
                    #        PVals[player]['bo'].append(float(sav[0])-birth_year)
                    #        PVals[player]['pos'].append(float(sav[14])/float(sav[4]))
                    #        PVals[player]['bbr'].append(float(sav[12])/float(sav[4]))

                    #    except:
                    #        # this is janky way to skip postseason. please fix this
                    #        #print 'Skipped for postseason',sav[0],sav[1]
                    #        pass

    return PVals


PVals = obtain_players(plist)



0 Rougned Odor
50 James McCann
100 Tommy Joseph
150 Max Stassi
200 Nori Aoki
250 Marcell Ozuna
300 Derek Fisher
350 Eric Hosmer
400 Tuffy Gosewisch
450 Kole Calhoun
500 Byron Buxton
550 Tomas Telis


In [60]:
from collections import Counter

poslist = ['C','1B','2B','3B','SS','CF','LF','RF','DH']


#print np.where(np.array(cntr.values()) > 50)[0][0]
#print cntr.keys()[np.where(np.array(cntr.values()) > 50)[0][0]]
#print PVals


for entry in PVals.keys():
    
    cntr = Counter(PVals[entry]['pos'])


    
    try:
        almost_pos = np.where((np.array(cntr.values()) > 5) & (np.array(cntr.values()) < 15 ))[0]
        
        for val in almost_pos:
                        
            if (cntr.keys()[val] != 'DH') & (cntr.keys()[val] != 'PR') & (cntr.keys()[val] != 'PH') & (np.sum(cntr.values()) > 50):
                print('{0:20s}{1:4s}{2:2d}'.format(entry,cntr.keys()[val],np.array(cntr.values())[val]))
                
        
    except:
        
        pass

Albert Pujols       1B   6
Kelby Tomlinson     SS   9
Kelby Tomlinson     3B   9
Jose Reyes          2B  10
Wilmer Flores       2B  10
Carlos Beltran      OF  12
Yangervis Solarte   SS  10
Yangervis Solarte   3B  11
Cliff Pennington    SS   9
Cliff Pennington    3B   9
Eduardo Nunez       SS  11
Brandon Belt        OF  11
Taylor Motter       OF  11
Taylor Motter       1B   8
Taylor Motter       2B  13
Jace Peterson       OF  12
Jace Peterson       1B   6
Jace Peterson       2B  12
Jace Peterson       3B  11
Daniel Descalso     3B   7
Daniel Descalso     1B   8
Alex Avila          1B  14
Matt Carpenter      2B  11
Trevor Plouffe      1B   7
Marwin Gonzalez     2B   9
Marwin Gonzalez     3B  14
Hanley Ramirez      1B  14
Ryan Goins          3B   6
Kendrys Morales     1B  10
Scooter Gennett     3B   9
Scooter Gennett     OF  13
Jose Martinez       1B   9
Tommy La Stella     3B  10
Tommy La Stella     2B  10
Arismendy Alcantara OF  11
Arismendy Alcantara 2B   9
Josh Harrison       OF   6
A